In [ ]:
#### Import Dependencies ####
import nemo.collections.asr as nemo_asr
from nemo.utils import exp_manager
import pytorch_lightning
from omegaconf import OmegaConf, open_dict
from nemo.core import adapter_mixins
from pytorch_lightning import Trainer
from nemo.collections.common.parts.adapter_modules import LinearAdapterConfig
import torch

from pytorch_lightning.loggers import WandbLogger

config_path = '/home/saurabh/nemo/NeMo/examples/asr/conf/wav2vec_ctc/wav2vecCTC.yaml'
params = OmegaConf.load(config_path)

In [ ]:
print(OmegaConf.to_yaml(params))

In [ ]:
accelerator = 'gpu' if torch.cuda.is_available() else 'cpu'
# accelerator = 'cpu'
max_steps = -1
max_epochs = 100

trainer = Trainer(devices=1, accelerator=accelerator, max_steps=max_steps,
                  enable_checkpointing=False, logger=False,
                  log_every_n_steps=5, check_val_every_n_epoch=3, 
                  max_epochs=max_epochs)


In [ ]:
train_manifest = '/home/saurabh/manifest/new/kanada_labelled/train.json'
test_manifest  = '/home/saurabh/manifest/new/kanada_labelled/valid.json' 

# Update paths to dataset
params.model.train_ds.manifest_filepath = train_manifest
params.model.train_ds.batch_size = 8
params.model.validation_ds.manifest_filepath = test_manifest
params.model.validation_ds.batch_size = 8
params.model.tokenizer.dir = '/home/saurabh/manifest/new/kanada_labelled'
params.model.tokenizer.type = 'wpe'

In [ ]:
print(OmegaConf.to_yaml(params))

In [ ]:
exp_direc = '/home/saurabh/experiments/kanada_w2v2_wpe'
name_exp = 'kanada'
#wandb Kargs
name_wandb = 'run01'
project_wandb = 'kanada_w2v2'

exp_config = exp_manager.ExpManagerConfig(
    exp_dir=exp_direc,
    name=name_exp,
    
    checkpoint_callback_params=exp_manager.CallbackParams(
        monitor='val_wer',
        mode='min',
        # always_save_nemo=True,
        save_best_model=False,
    ),
    
    resume_if_exists=True,
    resume_ignore_no_checkpoint=True,
    create_tensorboard_logger= False,
    create_checkpoint_callback= True,
  
    create_wandb_logger=True,
    wandb_logger_kwargs={
        'name':name_wandb, 
        'project':project_wandb
    },
)

exp_config=OmegaConf.structured(exp_config)
logdir = exp_manager.exp_manager(trainer, exp_config)

In [ ]:
model =  nemo_asr.models.EncDecCTCModelBPE(cfg=params.model, trainer=trainer)

In [ ]:
trainer.fit(model)